In [1]:
from cryptography.exceptions import InvalidSignature
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization

In [2]:
def generate_key():
    private_key = rsa.generate_private_key(public_exponent=65537,key_size=2048)
    public_key = private_key.public_key()
    #Before serializing, cryptographic keys are typically represented as objects in memory
    private_pem = private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption())
    public_pem = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo)

    return private_pem, public_pem

In [3]:
private,public = generate_key()

with open('private.key','wb') as file:
    file.write(private)

with open('public.key','wb') as file:
    file.write(public)

In [4]:
with open('private.key','rb') as private:
    private_key = serialization.load_pem_private_key(private.read(),password=None)

with open('public.key','rb') as public:
    public_key = serialization.load_pem_public_key(public.read())

In [5]:
def signature(message,private_key):
    # hash the message first
    hash_algo = hashes.Hash(hashes.SHA256())
    hash_algo.update(message)
    hashed_message = hash_algo.finalize()

    # sign the hashed message

    sign = private_key.sign(hashed_message,
    padding.PSS(mgf= padding.MGF1(hashes.SHA256()),
    salt_length = padding.PSS.MAX_LENGTH),
    hashes.SHA256() )

    return sign

In [6]:
def verify_signature(message,sign,public_key):
    hash_algo = hashes.Hash(hashes.SHA256())
    hash_algo.update(message)
    hashed_message = hash_algo.finalize()

    try:
        public_key.verify(sign,hashed_message,
        padding.PSS(mgf = padding.MGF1(hashes.SHA256()),salt_length=padding.PSS.MAX_LENGTH)
        ,hashes.SHA256())

        print('Signature is Valid')


        return True

    except InvalidSignature:
        print('Sign not valid')
        return False

In [7]:
# Generate the signature
message = b'Hello, this is secure message'
sign = signature(message,private_key)

In [8]:
# Verify signature
output = verify_signature(message,sign,public_key)

Signature is Valid


In [9]:
# When public key is not correct:
wrong_private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
wrong_public_key = wrong_private_key.public_key()
output = verify_signature(message,sign,wrong_public_key)

Sign not valid
